# Server

In [ ]:
import asyncio

def run_server(host, port):
    
    loop = asyncio.get_event_loop()
    coro = loop.create_server(
        ClientServerProtocol,
        host, port
    )

    server = loop.run_until_complete(coro)

    loop.run_forever()

    server.close()
    loop.run_until_complete(server.wait_closed())
    loop.close()
    

class ClientServerProtocol(asyncio.Protocol):
    
    data ={}
    
    def connection_made(self, transport):
        self.transport = transport

    def data_received(self, data):
        resp = self.process_data(data.decode())
        self.transport.write(resp.encode())

    def process_data(self, request):
        
        response = 'ok\n'
        
        try:
            request = request.split()
            
            assert(len(request) in[2, 4])
            
            
            if len(request) == 2:
                assert(request[0] == 'get')
            
                if request[1] == '*':
                    keys = self.data.keys()
                elif request[1] in self.data.keys():
                    keys = [request[1]]
                else:
                    keys = []

                for key in keys:
                    for record in self.data[key]:
                        value, timestamp = record[0], record[1]
                        response += f'{key} {value} {timestamp}\n'       
            else:
                assert(request[0] == 'put')
                
                key, value, timestamp = request[1:4]
                record = (value, timestamp)
                
                if key not in self.data.keys():
                    self.data[key] = [record]
                elif record not in self.data[key]:
                    self.data[key].append(record)
        except:
            response = 'error\nwrong command\n'
            
        response += '\n'
        
        return response

# Client

In [ ]:
import socket
import time

class ClientError(Exception):
    pass

class Client:
    def __init__(self, host, port, timeout=None):
        self.host = host
        self.port = port 
        self.timeout = timeout
        
        try:
            self.conn = socket.create_connection((self.host, self.port), self.timeout)
        except:
            raise ClientError()
        
    def _listen(self):
        response = b''
        try:
            while not response.endswith(b'\n\n'):
                response += self.conn.recv(1024)
        except:
            raise ClientError()
        response = response.decode()
            
        
        responce_type, data = response.split('\n', 1)
        if responce_type == 'error':
            raise ClientError()
            
        return data.strip()
        
    def put(self, key, value, timestamp=str(int(time.time()))):
        try:
            self.conn.sendall(f'put {key} {value} {timestamp}\n'.encode())
        except:
            raise ClientError()
            
        self._listen()
    
    def get(self, name):
        try:
            self.conn.sendall(f'get {name}\n'.encode())
        except:
            raise ClientError()
            
        data = self._listen()

        answer_dict = {}
        
        if data == '':
            return answer_dict
        try:
            for record in data.split('\n'):
                key, value, ts = record.split()
                if key in answer_dict.keys():
                    answer_dict[key].append((int(ts), float(value)))
                else:
                    answer_dict[key] = [(int(ts), float(value))]
        except:
            raise ClientError()
        
        return answer_dict